# 🎶 Predicting Lyrics
<hr>
<br>
<b> Group name: <strike>AA</strike> <i>My Machine Learning Romance</i> </b> <br>
- Fiorella Tenorio <br>
- Katja Wittfoth <br>
- Rebecca Reilly <br>
- Victoria Suarez <br>
- Viviana Márquez <br>

<br>
<img src='USF.png', style="width:200px;" align="left">
Friday, December 7th, 2018. <br>
<b> MSDS 621 </b> - Introduction to Machine Learning 

## Goal 
<center>
    <img src="are you smarter.png">
    <big>Predict genre based on lyrics</big><br><br>
    🥊 Supervised <i>vs.</i> Unspervised  <i>vs.</i> Humans 🥊
</center> 

## Workflow Pipeline
<center>
    <img src="workflow2.png" style="height:500px">
</center> 

In [1]:
# Libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction import stop_words
from sklearn.model_selection import GridSearchCV


import glob
import os
import pandas as pd
import matplotlib.pyplot as plt   
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mlxtend.classifier import EnsembleVoteClassifier

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score


import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
df = pd.read_csv('/Users/vivianamarquez/Desktop/USF/3. Fall 18/Machine Learning/ML Project/lyrics.csv')

In [3]:
# Clean data
def clean_lyrics(text):
    text = re.sub('\n', ' ', text)  # removes new lines
    text = re.sub('\d', '', text)  # removes numbers
    text = re.sub('\t', '', text)  # removes tabs
    words = text.split(" ")
    words = [w for w in words if len(w) > 2]  # removes a, an, to, at, be, ...
    return ' '.join(words)


df = df[pd.notnull(df['lyrics'])]
df = df[df['genre'] != 'Not Available']
df = df[df['genre'] != 'Other']
df['lyrics_clean'] = df.lyrics.apply(clean_lyrics)

df_clean = df[(df.genre == 'Country') | (df.genre == 'Metal') |
              (df.genre == 'Hip-Hop') | (df.genre == 'Jazz') | (df.genre == 'Electronic')]

df_clean = df_clean.sample(frac=1)  # shuffle

## 📊 1. Get data and pre-process

- 380,000+ lyrics  from MetroLyrics in <a href="https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics">Kaggle</a>

#### 🛁 Cleaning 
- Remove: `Null` lyrics, "Not available" genre, punctuation, numbers, symbols, spaces, stop words.
- Shuffle data.
- **Genres**: Country, Metal, Hip-Hop, Jazz, Electronic.

In [4]:
df_clean.head()

,index,song,year,artist,genre,lyrics,lyrics_clean
346876,346876,secret-track,2014,childish-gambino,Hip-Hop,No matter what you say or what you do\nWhen I'...,"matter what you say what you When I'm alone, I..."
286644,286644,futuristic-space-doubleswee-rap-in-the-galaxy,2014,doubleswee,Hip-Hop,"You, Doubleswee\nIn space\nWhat up guys\nYeah ...","You, Doubleswee space What guys Yeah it's Chil..."
166706,166706,just-one-of-those-things,2014,django-reinhardt,Jazz,It was just one of those things\nJust one of t...,was just one those things Just one those crazy...
336273,336273,gimme-half,2009,the-devil-wears-prada,Metal,"Keep this on your mind,\nKeep it within your e...","Keep this your mind, Keep within your eyelids...."
53140,53140,horus-rises,2010,at-the-throne-of-judgement,Metal,"Rise.\nA sentence given, to the afflicted livi...","Rise. sentence given, the afflicted living, it..."


## 👠 2. Modeling

### ✂️ Splitting data

In [6]:
train, test = train_test_split(df_clean, test_size=0.2)
train_lyrics = train['lyrics']
train_genre = train['genre']
test_lyrics = test['lyrics']
test_genre = test['genre']

### 🥊 Supervised model: Multinomial Naive-Bayes

In [9]:
# Pipelines
pipe_lr_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('lr', LogisticRegression())
])

pipe_lr_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('lr', LogisticRegression())
])

pipe_nb_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('nb', MultinomialNB())
])

pipe_nb_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('nb', MultinomialNB())
])

pipe_svc_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())
])

pipe_svc_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())
])

pipelines = [pipe_lr_cv, pipe_lr_tf, pipe_nb_cv, pipe_nb_tf, pipe_svc_cv, pipe_svc_tf]

In [10]:
# Fit pipe
for pipe in pipelines:
    pipe.fit(train_lyrics, train_genre)

In [16]:
methods = []

for pipe in pipelines:
    name = pipe.steps[-1][1].__class__.__name__.split('.')[-1]
    vect = pipe.steps[0][1].__class__.__name__.split('.')[-1]
    
    predicted = pipe.predict(test_lyrics)
    accuracy = accuracy_score(predicted,test['genre'])
    conf_mat = confusion_matrix(test['genre'], predicted)
    
    methods.append([name,vect,accuracy,conf_mat])

methods=pd.DataFrame(methods)
methods.columns = ['Model', 'Vectorizer', 'Accuracy', "Confussion_Matrix"]
methods["Strategy"] = methods["Model"]+ " -- " + methods["Vectorizer"]
methods.set_index(["Strategy"],inplace=True)

In [17]:
methods

,Model,Vectorizer,Accuracy,Confussion_Matrix
Strategy,,,,
LogisticRegression -- CountVectorizer,LogisticRegression,CountVectorizer,0.773675,"[[2278, 96, 86, 228, 174], [198, 516, 253, 107..."
LogisticRegression -- TfidfVectorizer,LogisticRegression,TfidfVectorizer,0.768797,"[[2288, 58, 125, 156, 235], [207, 400, 312, 75..."
MultinomialNB -- CountVectorizer,MultinomialNB,CountVectorizer,0.732691,"[[2599, 18, 74, 38, 133], [535, 358, 286, 33, ..."
MultinomialNB -- TfidfVectorizer,MultinomialNB,TfidfVectorizer,0.630012,"[[1086, 10, 1301, 1, 464], [52, 50, 950, 2, 57..."
LinearSVC -- CountVectorizer,LinearSVC,CountVectorizer,0.733135,"[[2098, 165, 109, 320, 170], [188, 632, 243, 1..."
LinearSVC -- TfidfVectorizer,LinearSVC,TfidfVectorizer,0.784570,"[[2297, 99, 91, 211, 164], [183, 612, 271, 93,..."


In [18]:
grid_params = dict(clf__alpha = np.arange(start=0, stop = 0.25, step = 0.05))

gs = GridSearchCV(estimator=pipe_svc_tf,  
                  param_grid=grid_params,
                  scoring='accuracy',
                  cv=10)

gs.fit(train_lyrics, train_genre)

ValueError: Invalid parameter clf for estimator Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
gs.best_params_

In [ ]:
gs.best_estimator_.steps[0][1]  # best model

In [ ]:
pred = gs.best_estimator_.predict(test_lyrics)

# Recap ✍️